In [1]:
import numpy as np 
import pandas as pd
import re

In [2]:
def clean_text(text):
    if isinstance(text, str): 
        return re.sub(r'\s+', ' ', text).strip()
    return text

In [3]:
def summary(df):
    print(f'data shape: {df.shape}')  
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df)
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
#     summ['min'] = desc['min'].values
#     summ['max'] = desc['max'].values
    return summ

In [22]:
# Function to replace codes start with per_
def replace_codes(df1, df2, column="Code"):
    replacements = dict(zip(df2["Old Code"], df2["Code"]))
    df1[column] = df1[column].replace(replacements)
    return df1



In [6]:
data = pd.read_excel('../data/main/main_data.xlsx')

add_metadata = pd.read_excel('../data/main/additional_metadata.xlsx')

changed_codes_social = pd.read_excel('../data/changed_codes_social.xlsx')



In [9]:
data.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Tunisia,TUN,"Intentional homicides (per 100,000 people)",VC.IHR.PSRC.P5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.738859e+00,4.621056e+00,NaN,NaN,NaN
1,Tunisia,TUN,"Internally displaced persons, new displacement...",VC.IDP.NWDS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.200000e+02,NaN,9.900000e+02,3.300000e+03,3.200000e+01,1.000000e+04,1.200000e+03,2.000000e+03,2.600000e+03
2,Tunisia,TUN,"Voice and Accountability: Percentile Rank, Upp...",VA.PER.RNK.UPPER,NaN,NaN,NaN,NaN,NaN,NaN,...,5.960591e+01,6.108374e+01,6.354680e+01,5.763547e+01,5.970874e+01,6.038647e+01,5.990338e+01,5.700483e+01,4.444444e+01,4.607843e+01
3,Tunisia,TUN,Voice and Accountability: Estimate,VA.EST,NaN,NaN,NaN,NaN,NaN,NaN,...,1.928789e-01,2.413065e-01,3.036524e-01,1.619458e-01,1.914097e-01,2.663486e-01,2.837476e-01,1.099035e-01,-2.481002e-01,-2.128970e-01
4,Tunisia,TUN,High-technology exports (current US$),TX.VAL.TECH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.714783e+08,8.384366e+08,8.710672e+08,8.508289e+08,8.289296e+08,8.405061e+08,8.326070e+08,9.596790e+08,1.041437e+09,1.198528e+09


In [18]:
data.shape

(1496, 68)

In [15]:
data.columns

Index([  'Country Name',   'Country Code', 'Indicator Name', 'Indicator Code',
                   1960,             1961,             1962,             1963,
                   1964,             1965,             1966,             1967,
                   1968,             1969,             1970,             1971,
                   1972,             1973,             1974,             1975,
                   1976,             1977,             1978,             1979,
                   1980,             1981,             1982,             1983,
                   1984,             1985,             1986,             1987,
                   1988,             1989,             1990,             1991,
                   1992,             1993,             1994,             1995,
                   1996,             1997,             1998,             1999,
                   2000,             2001,             2002,             2003,
                   2004,             2005,          

In [16]:
data = data.rename(columns={'Indicator Code': 'Code'})

In [19]:
summary(data).style.background_gradient(cmap='Blues')

data shape: (1496, 68)


,data type,#missing,%missing,#unique
Country Name,object,0,0.000000,1
Country Code,object,0,0.000000,1
Indicator Name,object,0,0.000000,1496
Code,object,0,0.000000,1496
1960,float64,1357,0.907086,131
1961,float64,1316,0.879679,171
1962,float64,1296,0.866310,193
1963,float64,1286,0.859626,203
1964,float64,1284,0.858289,205
1965,float64,1188,0.794118,301


In [20]:
data.drop(columns=['Country Name', 'Country Code'], inplace=True)


In [23]:
data = replace_codes(data, changed_codes_social)


In [25]:
data[data['Code'].str.startswith('per_')]

,Indicator Name,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023


In [37]:
year_cols = data.columns[2:]

missing_codes = data[data[year_cols].isna().all(axis=1)]

constant_codes = data[(data[year_cols].nunique(axis=1, dropna=True) == 1)]

In [38]:
missing_codes

,Indicator Name,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
51,Vitamin A supplementation coverage rate (% of ...,SN.ITK.VITA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,"Child employment in services, female (% of fem...",SL.SRV.0714.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,Child employment in manufacturing (% of econom...,SL.MNF.0714.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,CPIA trade rating (1=low to 6=high),IQ.CPA.TRAD.XQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,CPIA public sector management and institutions...,IQ.CPA.PUBS.XQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,"Net official flows from UN agencies, UNIDO (cu...",DT.NFL.UNIDO.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1383,"Net official flows from UN agencies, UNECE (cu...",DT.NFL.UNEC.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,"Incidence of HIV, all (per 1,000 uninfected po...",SH.HIV.INCD.TL.P3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1432,Women who believe a husband is justified in be...,SG.VAW.NEGL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
constant_codes

,Indicator Name,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
54,"Children in employment, wage workers, female (...",SL.WAG.0714.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,"Part time employment, male (% of total male em...",SL.TLF.PART.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.35,NaN,NaN,NaN,NaN
69,"Children in employment, total (% of children a...",SL.TLF.0714.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,"Average working hours of children, working onl...",SL.TLF.0714.WK.MA.TM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,"Average working hours of children, study and w...",SL.TLF.0714.SW.TM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,"Trained teachers in upper secondary education,...",SE.SEC.TCAQ.UP.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,100.0,100.0,NaN,100.0,NaN,100.0,NaN
1445,"Trained teachers in lower secondary education,...",SE.SEC.TCAQ.LO.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,100.0,100.0,NaN,100.0,NaN,100.0,NaN
1457,"Trained teachers in primary education, male (%...",SE.PRM.TCAQ.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,100.0,100.0,100.0,100.0,NaN,100.0,NaN,100.0,100.0
1467,"Trained teachers in preprimary education, fema...",SE.PRE.TCAQ.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0


In [41]:
codes_to_remove = set(missing_codes["Code"]).union(set(constant_codes["Code"]))

data = data[~data["Code"].isin(codes_to_remove)]

In [45]:
data.to_excel("../data/main/cleaned_main_data.xlsx", index=False)

In [44]:
codes_to_remove_df = pd.DataFrame(list(codes_to_remove), columns=["Code"])  
codes_to_remove_df.to_excel("../data/codes_to_remove.xlsx", index=False)

In [54]:
year_cols = data.columns[2:]

data_long = data.melt(id_vars=["Code", "Indicator Name"], 
                      value_vars=year_cols, 
                      var_name="Year", 
                      value_name="Value").dropna().reset_index(drop=True)   

In [58]:
data_long = data_long.sort_values(['Code','Year'])

In [59]:
data_long

,Code,Indicator Name,Year,Value
314,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,1961,23.181818
510,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,1962,29.703482
716,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,1963,23.221894
924,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,1964,20.685326
1228,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,1965,15.480304
...,...,...,...,...
30507,VC.IHR.PSRC.P5,"Intentional homicides (per 100,000 people)",2009,2.364504
31504,VC.IHR.PSRC.P5,"Intentional homicides (per 100,000 people)",2010,2.625042
33606,VC.IHR.PSRC.P5,"Intentional homicides (per 100,000 people)",2012,2.971081
40490,VC.IHR.PSRC.P5,"Intentional homicides (per 100,000 people)",2019,4.738859


In [61]:
data_long.to_excel("../data/main/long_main_data.xlsx", index=False)